In [15]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Calculate Distances") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [16]:
dfDist = spark.read.parquet("../processeddata/driv_dist.parquet")
dfDist.printSchema()

root
 |-- UATYPE: string (nullable = true)
 |-- ddist: double (nullable = true)
 |-- dist: double (nullable = true)
 |-- id: string (nullable = true)
 |-- m_house_unit: long (nullable = true)
 |-- m_id: long (nullable = true)
 |-- m_land: double (nullable = true)
 |-- m_lat_d: double (nullable = true)
 |-- m_lat_r: double (nullable = true)
 |-- m_long_d: double (nullable = true)
 |-- m_long_r: double (nullable = true)
 |-- m_pop: long (nullable = true)
 |-- m_water: double (nullable = true)
 |-- name: string (nullable = true)
 |-- z_house_unit: long (nullable = true)
 |-- z_id: long (nullable = true)
 |-- z_land: double (nullable = true)
 |-- z_lat_d: double (nullable = true)
 |-- z_lat_r: double (nullable = true)
 |-- z_long_d: double (nullable = true)
 |-- z_long_r: double (nullable = true)
 |-- z_pop: long (nullable = true)
 |-- z_water: double (nullable = true)



In [17]:
dfIncomesRaw = spark.read.load("../rawdata/ACS_15_5YR_S1903/ACS_15_5YR_S1903b.csv", format="csv", delimiter=",", header=True, inferSchema=True)
dfIncomesRaw.printSchema()
dfIncomesRaw.count()

root
 |-- GEOid: string (nullable = true)
 |-- GEOid2: integer (nullable = true)
 |-- GEOdisplaylabel: string (nullable = true)
 |-- HC01_EST_VC02: integer (nullable = true)
 |-- HC01_MOE_VC02: integer (nullable = true)
 |-- HC02_EST_VC02: integer (nullable = true)
 |-- HC02_MOE_VC02: integer (nullable = true)
 |-- HC01_EST_VC04: double (nullable = true)
 |-- HC01_MOE_VC04: double (nullable = true)
 |-- HC02_EST_VC04: integer (nullable = true)
 |-- HC02_MOE_VC04: integer (nullable = true)
 |-- HC01_EST_VC05: double (nullable = true)
 |-- HC01_MOE_VC05: double (nullable = true)
 |-- HC02_EST_VC05: integer (nullable = true)
 |-- HC02_MOE_VC05: integer (nullable = true)
 |-- HC01_EST_VC06: double (nullable = true)
 |-- HC01_MOE_VC06: double (nullable = true)
 |-- HC02_EST_VC06: integer (nullable = true)
 |-- HC02_MOE_VC06: integer (nullable = true)
 |-- HC01_EST_VC07: double (nullable = true)
 |-- HC01_MOE_VC07: double (nullable = true)
 |-- HC02_EST_VC07: integer (nullable = true)
 |-- H

33120

In [18]:
dfIncomes = dfIncomesRaw.select('GEOid2','HC01_EST_VC02','HC02_EST_VC02')\
                            .withColumnRenamed('GEOid2','z_id')\
                            .withColumnRenamed('HC01_EST_VC02','z_households')\
                            .withColumnRenamed('HC02_EST_VC02', 'z_med_inc')
dfIncomes = dfIncomes.select(dfIncomes.z_id.cast('long'),\
                             dfIncomes.z_households.cast('long'),\
                             dfIncomes.z_med_inc.cast('long'))
dfIncomes.printSchema()

root
 |-- z_id: long (nullable = true)
 |-- z_households: long (nullable = true)
 |-- z_med_inc: long (nullable = true)



In [19]:
dfFinancialsRaw = spark.read.load("../rawdata/ACS_15_5YR_DP03/ACS_15_5YR_DP03b.csv", format="csv", delimiter=",", header=True, inferSchema=True)
dfFinancialsRaw.printSchema()
dfFinancialsRaw.count()
dfFinancialsRaw.take(1)

root
 |-- GEOid: string (nullable = true)
 |-- GEOid2: integer (nullable = true)
 |-- GEOdisplay-label: string (nullable = true)
 |-- HC01_VC03: integer (nullable = true)
 |-- HC02_VC03: integer (nullable = true)
 |-- HC03_VC03: integer (nullable = true)
 |-- HC04_VC03: string (nullable = true)
 |-- HC01_VC04: integer (nullable = true)
 |-- HC02_VC04: integer (nullable = true)
 |-- HC03_VC04: double (nullable = true)
 |-- HC04_VC04: double (nullable = true)
 |-- HC01_VC05: integer (nullable = true)
 |-- HC02_VC05: integer (nullable = true)
 |-- HC03_VC05: double (nullable = true)
 |-- HC04_VC05: double (nullable = true)
 |-- HC01_VC06: integer (nullable = true)
 |-- HC02_VC06: integer (nullable = true)
 |-- HC03_VC06: double (nullable = true)
 |-- HC04_VC06: double (nullable = true)
 |-- HC01_VC07: integer (nullable = true)
 |-- HC02_VC07: integer (nullable = true)
 |-- HC03_VC07: double (nullable = true)
 |-- HC04_VC07: double (nullable = true)
 |-- HC01_VC08: integer (nullable = true

[Row(GEOid='8600000US00601', GEOid2=601, GEOdisplay-label='ZCTA5 00601', HC01_VC03=14262, HC02_VC03=215, HC03_VC03=14262, HC04_VC03=None, HC01_VC04=6162, HC02_VC04=363, HC03_VC04=43.2, HC04_VC04=2.6, HC01_VC05=6162, HC02_VC05=363, HC03_VC05=43.2, HC04_VC05=2.6, HC01_VC06=3949, HC02_VC06=345, HC03_VC06=27.7, HC04_VC06=2.4, HC01_VC07=2213, HC02_VC07=342, HC03_VC07=15.5, HC04_VC07=2.5, HC01_VC08=0, HC02_VC08=21, HC03_VC08=0.0, HC04_VC08=0.3, HC01_VC09=8100, HC02_VC09=428, HC03_VC09=56.8, HC04_VC09=2.6, HC01_VC11=6162, HC02_VC11=363, HC03_VC11=6162, HC04_VC11=None, HC01_VC12=None, HC02_VC12=None, HC03_VC12=35.9, HC04_VC12=4.8, HC01_VC14=7343, HC02_VC14=118, HC03_VC14=7343, HC04_VC14=None, HC01_VC15=2801, HC02_VC15=222, HC03_VC15=38.1, HC04_VC15=3.1, HC01_VC16=2801, HC02_VC16=222, HC03_VC16=38.1, HC04_VC16=3.1, HC01_VC17=1664, HC02_VC17=194, HC03_VC17=22.7, HC04_VC17=2.6, HC01_VC19=1172, HC02_VC19=121, HC03_VC19=1172, HC04_VC19=None, HC01_VC20=841, HC02_VC20=149, HC03_VC20=71.8, HC04_VC20=1

In [20]:
dfFinancials = dfFinancialsRaw.select('Geoid2','HC03_VC161','HC03_VC28')
dfFinancials = dfFinancials.withColumnRenamed('Geoid2','z_id')\
                            .withColumnRenamed('HC03_VC161','z_pov')\
                            .withColumnRenamed('HC03_VC28','z_per_comm')

In [21]:
MAX_DDIST = 65
COMM_PER = .25
BUS_DAYS = 250

dfFull=dfDist.join(dfIncomes,dfIncomes.z_id==dfDist.z_id).drop(dfIncomes.z_id).where(dfDist.ddist<MAX_DDIST)
#print(dfFull.count())
#print(dfFull.select(dfFull.z_id).distinct().count())

dfCommuteTo = dfFull.groupBy(dfFull.z_id).agg({"m_pop":"sum"}).orderBy(dfFull.z_id)\
                .withColumnRenamed('sum(m_pop)','z_m_tot')

dfFull2 = dfFull.join(dfCommuteTo,dfFull.z_id==dfCommuteTo.z_id).drop(dfCommuteTo.z_id)

dfFull3 = dfFull2.withColumn('z_comm',(dfFull2.z_pop*COMM_PER).cast('long'))\
                .withColumn('z_m_comm',(dfFull2.z_pop*COMM_PER*dfFull2.m_pop/dfFull2.z_m_tot).cast('long'))

#dfFull3.select('z_id','name','m_pop','z_pop','z_comm','z_m_comm').orderBy(dfFull3.z_m_comm.desc()).show(30)

dfFull3 = dfFull3.withColumn('z_m_tot_miles_yr',dfFull3.ddist*2*BUS_DAYS*dfFull3.z_m_comm)
#dfFull3.select('z_id','name','m_pop','z_pop','z_m_comm','z_m_tot_miles_yr').orderBy(dfFull3.z_m_comm.desc()).show(30)

In [22]:
dfFull3.printSchema()

root
 |-- UATYPE: string (nullable = true)
 |-- ddist: double (nullable = true)
 |-- dist: double (nullable = true)
 |-- id: string (nullable = true)
 |-- m_house_unit: long (nullable = true)
 |-- m_id: long (nullable = true)
 |-- m_land: double (nullable = true)
 |-- m_lat_d: double (nullable = true)
 |-- m_lat_r: double (nullable = true)
 |-- m_long_d: double (nullable = true)
 |-- m_long_r: double (nullable = true)
 |-- m_pop: long (nullable = true)
 |-- m_water: double (nullable = true)
 |-- name: string (nullable = true)
 |-- z_house_unit: long (nullable = true)
 |-- z_land: double (nullable = true)
 |-- z_lat_d: double (nullable = true)
 |-- z_lat_r: double (nullable = true)
 |-- z_long_d: double (nullable = true)
 |-- z_long_r: double (nullable = true)
 |-- z_pop: long (nullable = true)
 |-- z_water: double (nullable = true)
 |-- z_households: long (nullable = true)
 |-- z_med_inc: long (nullable = true)
 |-- z_id: long (nullable = true)
 |-- z_m_tot: long (nullable = true)
 |--

In [23]:
FUEL_ECON = 21.4 #MPG
CARB_PER_GAL = 0.008887
MILES2CARB = 1/FUEL_ECON*CARB_PER_GAL
PERSONSPERHH = 4

# Normalize Data
dfZData = dfFull3.select('z_id','z_lat_d','z_long_d','z_land','z_water','z_pop','z_households','z_comm','z_med_inc','z_house_unit')\
                    .distinct()
dfTemp = dfFull3.groupBy('z_id').agg({'z_m_tot_miles_yr':'sum'})
dfZData = dfZData.join(dfTemp,dfZData.z_id==dfTemp.z_id).drop(dfTemp.z_id).withColumnRenamed('sum(z_m_tot_miles_yr)','z_comm_miles')

dfZData = dfZData.withColumn('z_comm_miles_ph',dfZData.z_comm_miles/dfZData.z_pop*PERSONSPERHH)
dfZData = dfZData.withColumn('z_carb_ton_ph',dfZData.z_comm_miles_ph*MILES2CARB)
dfZData = dfZData.join(dfFinancials,dfZData.z_id==dfFinancials.z_id).drop(dfZData.z_id)
#dfZData.orderBy(dfZData.z_carb_ton_ph.desc()).show()

In [24]:
dfZData.coalesce(1).write.csv("../processeddata/zcarbon_v2.csv",header=True)